In [1]:
# 이미지있으면 이미지처리 -> 관련키워드는 따로
# 자격요건 넣기
# 랜덤으로 timesleep 주기
# 막혔을때 예외처리
# 스킬 추가

In [80]:
# conda install beautifulsoup4
# conda install selenium
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import random
import re
import numpy as np

In [3]:
def remove_tags(string):
    return re.sub('<.+?>', '', str(string), 0).strip()

Data = pd.DataFrame({}, columns=[\
    'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url'])

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        ## Head-less 설정
#options.add_argument('--no-sandbox')
#options.add_argumetn('--blink-setting=imagesEnabled=false') # 이미지 로딩 안함
#options.add_argument('--nute-audio') # 음소거
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-site-isolation-trials")
#options.add_argument('--disable-setuid-sandbox')
options.add_argument('--incognito') # 시크릿모드
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36') # user와 같은 설정 주기
chromedriver = '../data/chromedriver'

In [7]:
#driver.quit()
driver = webdriver.Chrome(chromedriver, options=options) # 크롬 드라이버 설정

In [8]:
base_url = 'https://www.jobkorea.co.kr/'
driver.get(base_url)
search_job = driver.find_element('xpath', '//*[@id="stext"]')
word = '데이터 사이언티스트'
search_job.send_keys(word+Keys.ENTER)
time.sleep(5) # 랜덤으로
html = driver.page_source

In [9]:
page_soup = BeautifulSoup(html, 'html.parser')
recruit_items = page_soup.find_all('li', {'class':'list-post'}) # list-post / list-post active

In [151]:
print (recruit_items[20])

<li class="list-post" data-gino="41949277" data-gno="39033535" data-mem-sys="21390" data-mem-type="C">
<div class="post">
<div class="post-list-corp">
<a class="name" href="/Recruit/Co_Read/C/creduinsa" target="_blank" title="새창">㈜멀티캠퍼스</a>
<button class="btn-fav pseudo-icn-old heart dev_favor-21390" type="button">관심기업</button>
</div>
<div class="post-list-info">
<a class="title" href="/Recruit/GI_Read/39033535?Oem_Code=C1&amp;logpath=1" onclick="GA_Virtual('홈&gt;통합검색&gt;공고뷰', 'https://www.jobkorea.co.kr/virtual/Recruit/GI_Read/39033535?Oem_Code=C1&amp;logpath=1');" target="_blank">[비전공자 환영] 멀티캠퍼스X고용노동부 프로젝트형 국비지원교육 모집</a>
<p class="option">
<span class="exp">경력무관</span>
<span class="edu">학력무관</span>
<span>연수생/교육생</span>
<span>서울</span>
<span class="date">~08/14(일)</span>
</p>
</div>
<button class="btn-scrap pseudo-icn-old star dev_scrap-41949277" type="button">스크랩</button>
</div>
</li>


In [150]:
####### 각 채용공고당 진행
# for-loop start
import sys
import copy

# ii = 15
# if ii == 15:
for ii in range(20,len(recruit_items)):
    print ('start ',ii)
    recruits = recruit_items[ii].find('a', {'class':'title dev_view'})
    rec_url = recruits['href']
    base_url = 'https://www.jobkorea.co.kr'
    search_url = base_url + rec_url
    driver.get(search_url)
    stime = 1+2*random.random()
    time.sleep(stime)
    html_job = BeautifulSoup(driver.page_source, 'html.parser')

    # iframe 찾기
    iframe = html_job.find('iframe', {'title':'상세요강'}) # html 내의 document (iframe) 처리
    driver.switch_to.frame('gib_frame') # iframe으로 옮겨가기
    stime = 1+1*random.random()
    time.sleep(stime)
    html_iframe = BeautifulSoup(driver.page_source, 'html.parser')

    # title, job, company, url
    title = remove_tags(html_job.find('title'))
    job = word
    company = remove_tags(html_job.find('span', {'class':'coName'}))
    html_str = str(html_iframe)
    url = search_url

    # keywords and location
    keyword = ''
    loc = ''
    assoc_keyword = html_job.find('article', {'id':'artKeywordSearch'})
    for key in assoc_keyword.find_all('li'):
        if str(key).find('btnLocal') > -1:
            loc = remove_tags(key).replace('&gt;', '>')
            continue
        keyword += remove_tags(key) + ', '

    # rec type
    rec = ''
    apply_qual = remove_tags(html_job.find('dl', {'class':'tbList'})).replace('\n','')
    apply_qual1 = apply_qual.find('경력')
    apply_qual2 = apply_qual.find('학력')
    if apply_qual2 >= 0:
        rec = apply_qual[apply_qual1+2:apply_qual2].replace(' ','')
    else:
        rec = apply_qual[apply_qual1+2:].replace(' ','')

    # 상세 채용 정보 내용
    duties = ''
    skills = ''
    preference = ''
    welfare = ''

    
    content = html_iframe.find('div', {'class':'detailed-summary-contents'})
    img = html_iframe.find('div', {'class':'detailReadImg'})
    if str(content) != 'None':
#         table_template = html_iframe.find('div', {'id':'dev-template-v2-root'})
#         print (table_template)
#         #template_no = table_template['templateno']
#         if not 'templateno' in table_template.keys():
#         elif table_template['templateno'] == '12':
#             content_table = html_iframe.find('div', {'class':'recruitment'})
#             content_lines = content_table.find('div', {'class':'recruitment-item'})
#         elif table_template['templateno'] == '11':
#             pass

        ps = content.find_all('p')
        for ip in ps:
            # duties
            str_ip = remove_tags(ip)
            if '담당업무' in str(ip): # duties
                case = 1
            elif any([needs in str(ip) for needs in ['필수', '자격']]): # skills
                case = 2
            elif '우대' in str(ip): # preference
                case = 3
            elif any([i in str(ip) for i in ['복지', '복리']]): # welfare
                case = 4
            elif any([i in str(ip) for i in ['전형절차']]):
                case = -1
            try:
                if case == 1: # duties
                    duties += str_ip + '\n'
                elif case == 2: # skills
                    skills += str_ip + '\n'
                elif case == 3: # preference
                    preference += str_ip + '\n'
                elif case == 4: # welfare
                    welfare += str_ip + '\n'
                elif case == -1: # others
                    print ('other : ',str_ip)
            except NameError:
                pass
    elif str(img) != 'None':
        pass

    # Data에 저장
    length = Data.shape[0]
    Data.loc[length] = [title, job, company, duties, skills, preference, welfare, loc, rec, url, keyword]
    Data

start  20
None


TypeError: 'NoneType' object is not subscriptable

In [147]:
import openpyxl
Data.to_excel('../output/DataScience_0721_4.xlsx')

In [145]:
print (url)
Data

https://www.jobkorea.co.kr/Recruit/GI_Read/38861523?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=20


,title,job,company,duties,skills,preference,welfare,loc,rec type,url,keyword
0,삼성웰스토리 채용 - B2B 온라인(E커머스) 및 Data 부문 경력사원 채용 (데...,데이터 사이언티스트,삼성웰스토리㈜,담당업무\nㆍ데이터 사이언티스트\n- 분석모델 개발(ML/DL) 검증 및 운영 기획...,"모집분야 및 자격요건\n데이터 사이언티스트\n자격요건\nㆍ알고리즘, 데이터 구조 등...","우대사항\nㆍ컴퓨터공학, AI, SW, 산업공학 등 관련 전공\nㆍAWS SageM...",,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr/Recruit/GI_Read/391...,"IT분야 직무채용관, 외식 직무채용관, 빅데이터·AI(인공지능), Python, 데..."
1,카카오뱅크 채용 - [빅데이터분석] 데이터 사이언티스트 (서비스분석) | 잡코리아,데이터 사이언티스트,카카오뱅크,담당업무\nㆍ추천/개인화 모형 개발\nㆍ고객 행동 예측 및 세그멘테이션\nㆍ전사 데...,모집분야 및 자격요건\n서비스분석( 0명 )\n필수 경험과 역량\nㆍ데이터 분석 및...,우대사항\nㆍ추천시스템/NLP/Graph Neural Network에 대한 이해가 ...,,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr/Recruit/GI_Read/391...,"금융·영업 직무채용관, IT분야 직무채용관, ERP·시스템분석·설계, 모델링, 빅데..."
2,카카오뱅크 채용 - [빅데이터분석] 데이터 사이언티스트 | 잡코리아,데이터 사이언티스트,카카오뱅크,담당업무\nㆍ머신러닝을 이용한 이상거래 탐지 모형 개발\nㆍ머신러닝을 활용한 FDS...,모집분야 및 자격요건\n데이터 사이언티스트( 0명 )\n필수 경험과 역량\nㆍ머신러...,우대사항\nㆍ금융사기 방지를 위한 이상탐지 분석 및 모형 개발 경험이 있는 분\nㆍ...,,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr/Recruit/GI_Read/390...,"금융·영업 직무채용관, IT분야 직무채용관, DBA·데이터베이스, SQL, 빅데이터..."
3,휴머스온 채용 - [데이터엔지니어] [데이터사이언티스트] [데이터 분석가] 경력 모...,데이터 사이언티스트,주식회사 휴머스온,,,,,서울 > 서울전지역,경력(2년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/389...,"이달의 우수 중소기업 일자리, 강소기업 채용관, IT분야 직무채용관, 빅데이터·AI..."
4,마이셀럽스 채용 - [마이셀럽스] 데이터 사이언티스트 정규직 채용 | 잡코리아,데이터 사이언티스트,(주)마이셀럽스,담당업무\nㆍ자연어 기반 추천 서비스를 위한 알고리즘 모델링\nㆍ딥러닝 모델의 최적...,모집분야 및 자격요건\n\n\n\n\n\n\n\n\n\n데이터 사이언티스트\n( ○...,"우대사항\nㆍ머신러닝, 통계 관련 석사 / 박사\nㆍTransformer / BER...","복리후생\nㆍ연차 상관 없는 높은 연봉 상승율, 스톡옵션\nㆍ수평적인 문화 제도 (...",서울 > 강남구,경력(3년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/388...,"IT분야 직무채용관, 빅데이터·AI(인공지능), 자연어처리(NLP), 데이터분석, ..."
5,카비 채용 - [CARVI] 데이터사이언티스트 총괄 | 잡코리아,데이터 사이언티스트,㈜카비,,,,,서울 > 금천구,경력(7년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/387...,"우수벤처기업 공동채용관, IT분야 직무채용관, DBA·데이터베이스, 데이터마이닝, ..."
6,카비 채용 - [CARVI] 데이터사이언티스트 (경력) | 잡코리아,데이터 사이언티스트,㈜카비,,,,,서울 > 금천구,경력,https://www.jobkorea.co.kr/Recruit/GI_Read/382...,"우수벤처기업 공동채용관, IT분야 직무채용관, DBA·데이터베이스, 데이터마이닝, ..."
7,레디엔터테인먼트 채용 - 역량있는 데이터사이언티스트를 모집합니다. | 잡코리아,데이터 사이언티스트,㈜레디엔터테인먼트,,,,,서울 > 강남구,경력(3년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/390...,"일학습병행 채용관, IT분야 직무채용관, 빅데이터·AI(인공지능), 데이터분석, 빅..."
8,비쥬웍스 채용 - 바이오 AI 데이터 사이언티스트 경력/신입 모집 | 잡코리아,데이터 사이언티스트,㈜비쥬웍스,담당업무\nㆍ 안과분야 의료 인공지능 모델 개발\n\n,모집분야 및 자격요건\n모집분야명 ( 0명 )\n자격요건\nㆍ Computer Vi...,우대사항\nㆍAWS 사용 경험\nㆍ통계 / 컴퓨터공학 및 관련 유관 분야 학사를 소...,"복리후생 지원\nㆍ점심식대 별도 제공\nㆍ도서비, 교육비 지원\nㆍ명절 선물 및 송...",서울 > 서초구,신입·경력(3년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/391...,"이달의 우수 중소기업 일자리, IT분야 직무채용관, 빅데이터·AI(인공지능), 빅데..."
9,파이온코퍼레이션 채용 - 데이터 사이언티스트 채용 | 잡코리아,데이터 사이언티스트,파이온코퍼레이션,담당업무\nㆍComputer Vision/영상 데이터 분석 및 모델링 - 이미지내 ...,모집분야 및 자격요건\n데이터 사이언티스트 ( ○명 )\n자격요건\nㆍ학력 : 대졸...,우대사항\nㆍML 기본 알고리즘 사용및 작동 방식의 이해\nㆍ기본적인 data pr...,,서울 > 강남구,경력무관,https://www.jobkorea.co.kr/Recruit/GI_Read/391...,"IT분야 직무채용관, 빅데이터·AI(인공지능), 데이터분석, 네트워크·통신서비스,"


In [135]:
# Data초기화
Data = pd.DataFrame({}, columns=[\
    'title','job', 'company','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url','keyword'])

In [111]:
driver.quit()